In [4]:
import pandas as pd
import numpy as np
# data = pd.read_excel('dataset_laptop.xlsx')

In [37]:
data.head()

,budget,cpu,gpu_merk,gpu,ram,memory_type,company,screen_type,screen_resolution,weight,type_laptop,kebutuhan
0,sedang,i3,intel,hd graphics,4,ssd,hp,full hd,1920x1080,1.91,notebook,office
1,sedang,i3,intel,hd graphics,8,ssd,lenovo,ips panel full hd,1920x1080,1.70,notebook,office
2,tinggi,i7,amd,radeon,16,ssd + hdd,dell,ips panel full hd,1920x1080,2.80,notebook,gaming
3,sedang,i3,amd,radeon r,4,ssd,dell,full hd,1920x1080,2.20,notebook,office
4,tinggi,i7,intel,hd graphics,8,ssd,asus,full hd,1920x1080,1.25,ultrabook,office


In [31]:
for col in data.columns:
    if data[col].dtypes == 'object':
        data[col] = data[col].str.lower()

In [51]:
# print(data['kebutuhan'].dtypes)
print(data['gpu'].unique())

['hd graphics' 'radeon 5xx' 'radeon r' 'iris plus graphic' 'uhd graphic'
 'radeon rx' 'geforce gtx' 'geforce' 'radeon pro']


In [45]:
for i in range(len(data)):
    if data['gpu'][i] == 'radeon pro 555':
        data['gpu'][i] = 'radeon pro'
    if data['gpu'][i] == 'radeon':
        data['gpu'][i] = 'radeon 5xx'

C:\Users\bs129\AppData\Local\Temp/ipykernel_17576/594284764.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gpu'][i] = 'radeon 5xx'
C:\Users\bs129\AppData\Local\Temp/ipykernel_17576/594284764.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gpu'][i] = 'radeon pro'


In [47]:
data.to_excel('dataset_laptop.xlsx', index=False)


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder

In [56]:
column_trans = ColumnTransformer(remainder="passthrough", transformers=[('onehot',OneHotEncoder(), [0,1,2,3,5,6,7,8,10])], sparse_threshold=0
)

In [57]:
NB = GaussianNB()
nb_model_pipelines = Pipeline(steps=[
                       ('preprocessing', column_trans),
                       ('naive_bayes_model', NB)
                       ])
dt = DecisionTreeClassifier(criterion='entropy', random_state=24, max_depth=8, max_leaf_nodes=8)

dt_pipe = Pipeline(steps=[('preprocessing', column_trans),
                       ('dt_model', dt)
                       ])

knn = KNeighborsClassifier(n_neighbors=3, weights='distance', algorithm='kd_tree')

knn_pipeline = Pipeline(steps=[('preprocessing', column_trans),
                       ('knn', knn)
                       ])
GB = GradientBoostingClassifier(loss='exponential',criterion='friedman_mse', learning_rate=0.05, min_samples_leaf=8, random_state=10)
gb_pipeline = Pipeline(steps=[('preprocessing', column_trans),
                       ('gbdt', GB)
                       ])

In [18]:
df = pd.read_excel('dataset_kebutuhan_laptop_example.xlsx')

In [19]:
df = df.drop_duplicates()

In [21]:
df.drop('Unnamed: 0', inplace=True, axis='columns')
df.head()

,budget,cpu,gpu_merk,gpu,ram,memory_type,company,screen_type,screen_resolution(pixel),weights,type_laptop,kebutuhan(target)
0,sedang,i5,Intel,UHD Graphics,8,SSD,Acer,IPS Panel Full HD,1920x1080,1.60,ultrabook,Office
1,sedang,i3,Intel,HD Graphics,4,SSD,HP,Full HD,1920x1080,1.91,notebook,Office
2,sedang,i3,Intel,HD Graphics,8,SSD,Lenovo,IPS Panel Full HD,1920x1080,1.70,notebook,Office
3,tinggi,i7,AMD,Radeon 5xx,16,SSD + HDD,Dell,IPS Panel Full HD,1920x1080,2.80,notebook,Gaming
4,sedang,i3,AMD,Radeon R,4,SSD,Dell,Full HD,1920x1080,2.20,notebook,Office


In [22]:
from sklearn.preprocessing import LabelEncoder

In [26]:
encode = LabelEncoder()
target = df['kebutuhan(target)']
data = df.drop('kebutuhan(target)', axis='columns')

for col in data.columns:
    if df[col].dtypes == 'object':
        df[col] = encode.fit_transform(df[col])

In [27]:
data.head()

,budget,cpu,gpu_merk,gpu,ram,memory_type,company,screen_type,screen_resolution(pixel),weights,type_laptop
0,0,2,1,13,8,3,0,2,3,1.60,4
1,0,1,1,2,4,3,4,0,3,1.91,3
2,0,1,1,2,8,3,5,2,3,1.70,3
3,1,3,0,7,16,4,3,2,3,2.80,3
4,0,1,0,10,4,3,3,0,3,2.20,3


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=20)

In [30]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [32]:
predict = knn.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, predict)

c:\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


0.9

In [38]:
from sklearn.model_selection import cross_val_score, KFold

fold = KFold(n_splits=10)

cross_val_score(knn,data, target, cv=fold, scoring='accuracy').mean()

c:\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

0.96

In [45]:
!pip install -U scikit-learn

ERROR: Could not find a version that satisfies the requirement sklearn==1.1 (from versions: 0.0)
ERROR: No matching distribution found for sklearn==1.1
You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.
